In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
"""
Required attributes:

#Distillery
#Vintage
#Number of bottles produced originally
Format
Amount
Weather quality
Price history
Warnings (e.g. damage)
#Strenght             
#Whisky Type (single malt etc)
Reserve price (yes/no)
Concluded (yes/no)

"""

'\nRequired attributes:\n\n#Distillery\n#Vintage\n#Number of bottles produced originally\nFormat\nAmount\nWeather quality\nPrice history\nWarnings (e.g. damage)\n#Strenght             \n#Whisky Type (single malt etc)\nReserve price (yes/no)\nConcluded (yes/no)\n\n'

In [3]:
website_link = 'https://www.highlandwhiskyauctions.com/past-auctions'

In [4]:
html = urllib.request.urlopen(website_link).read()

In [5]:
soup = BeautifulSoup(html,'lxml')

In [6]:
#soup.prettify

In [7]:
#The total number of auctions available

available_auctions = soup.find_all('li',class_ = 'itemSmall')

In [8]:
auction_months = []
for i in available_auctions:
    auction_months.append(i.div.a['href'])

In [9]:
auction_months_links = {}
for i in auction_months:
    auction_months_links[i.split('/')[-1]] = 'https://www.highlandwhiskyauctions.com'+i

In [10]:
for i in auction_months_links.keys():
    print(auction_months_links[i])

https://www.highlandwhiskyauctions.com/february-2023
https://www.highlandwhiskyauctions.com/december-2022
https://www.highlandwhiskyauctions.com/november-2022
https://www.highlandwhiskyauctions.com/october-2022
https://www.highlandwhiskyauctions.com/september-2022
https://www.highlandwhiskyauctions.com/august-2022
https://www.highlandwhiskyauctions.com/july-2022
https://www.highlandwhiskyauctions.com/may-2022
https://www.highlandwhiskyauctions.com/april-2022
https://www.highlandwhiskyauctions.com/march-2022
https://www.highlandwhiskyauctions.com/february-2022
https://www.highlandwhiskyauctions.com/janurary-2022
https://www.highlandwhiskyauctions.com/december-2021


In [11]:
#Finding number of available pages in each auction

for i in auction_months_links.keys():
    auction_month_page = (auction_months_links[i])
    html = urllib.request.urlopen(auction_month_page).read()
    soup = BeautifulSoup(html,'lxml')

In [12]:
website = 'https://www.highlandwhiskyauctions.com/february-2023/page-1'
html = urllib.request.urlopen(website).read()
soup = BeautifulSoup(html,'lxml')

In [181]:
#soup.find_all('li')

In [14]:
#On observation, the webpages are dynamic in nature

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [36]:
auction_pages = list(auction_months_links.values())

In [77]:
#Finding number of pages assigned in each auction
last_page_dict = {}
for i in auction_pages:
    driver = webdriver.Chrome()
    driver.get(i)
    page_finder = driver.find_element(By.CLASS_NAME, 'pagination')
    sel = page_finder.get_attribute('innerHTML')
    soup = BeautifulSoup(sel,'lxml')
    last_page = soup.find_all('li')[-2].text
    last_page = int(last_page.replace('\n','').replace(' ',''))
    last_page_dict[i.split('/')[-1]] = last_page

In [78]:
last_page_dict

{'february-2023': 24,
 'december-2022': 24,
 'november-2022': 28,
 'october-2022': 18,
 'september-2022': 14,
 'august-2022': 16,
 'july-2022': 15,
 'may-2022': 13,
 'april-2022': 10,
 'march-2022': 10,
 'february-2022': 8,
 'janurary-2022': 4,
 'december-2021': 3}

In [79]:
auction_pages

['https://www.highlandwhiskyauctions.com/february-2023',
 'https://www.highlandwhiskyauctions.com/december-2022',
 'https://www.highlandwhiskyauctions.com/november-2022',
 'https://www.highlandwhiskyauctions.com/october-2022',
 'https://www.highlandwhiskyauctions.com/september-2022',
 'https://www.highlandwhiskyauctions.com/august-2022',
 'https://www.highlandwhiskyauctions.com/july-2022',
 'https://www.highlandwhiskyauctions.com/may-2022',
 'https://www.highlandwhiskyauctions.com/april-2022',
 'https://www.highlandwhiskyauctions.com/march-2022',
 'https://www.highlandwhiskyauctions.com/february-2022',
 'https://www.highlandwhiskyauctions.com/janurary-2022',
 'https://www.highlandwhiskyauctions.com/december-2021']

In [176]:
from tqdm import tqdm

In [180]:
whiskey_dict = {}
for x in tqdm(last_page_dict.keys()):
    for p in range(1,last_page_dict[x]+1):
        website = 'https://www.highlandwhiskyauctions.com/' + x + '/page-' + str(p)
        driver = webdriver.Chrome()
        driver.get(website)
        lot_finder_lst = driver.find_elements(By.CLASS_NAME, 'innerWrap')
        for lot in lot_finder_lst:
            sel = lot.get_attribute('innerHTML')
            soup = BeautifulSoup(sel,'lxml')
            link = soup.find('a')['href']
            outer_title = soup.find('a')['title']
            lot_outer_details = soup.find('div',class_ = 'itemDetails')
            lot_outer_details_lst = lot_outer_details.find_all('div')[:-3]
            whiskey_dict[link] = []
            whiskey_dict[link].append({'outer_lot_title':outer_title})
            for i in lot_outer_details_lst:
                detail = (i.text.replace('\n','').strip())
                if ':' in detail:
                    detail = detail.split(':')
                    whiskey_dict[link].append({detail[0]:detail[-1]})
                else:
                    detail = detail.split()
                    whiskey_dict[link].append({' '.join(detail[:-1]):detail[-1]})

100%|██████████████████████████████████████████| 13/13 [49:31<00:00, 228.54s/it]


In [246]:
len(whiskey_dict)

6529

In [257]:
for i in tqdm(whiskey_dict):
    website = i
    if True:
        html = urllib.request.urlopen(website).read()
        soup = BeautifulSoup(html,'lxml')
        lot_details = soup.find('ul',class_ = 'lotProperties')
        attributes = lot_details.find_all('li')
        available_attributes = []
        for i in attributes:
            available_attributes.append(i.h5.text)
        attributes_dict = {}
        available_attributes_values = []
        for j in attributes:
            available_attributes_values.append(j.div.text)
        lot_dict = {}
        for k in range(0,len(available_attributes)):
            lot_dict[available_attributes[k]] = available_attributes_values[k].replace(available_attributes[k],'').replace('\n','').replace(' ','')
        lot_header = soup.find('div',class_ = 'itemHeader')
        lot_dict['Name'] = lot_header.h1.text
        lot_header['Lot_id'] = lot_header.h2.text
        warnings = soup.find('div',class_ = 'pleaseNote')
        try:
            warnings = warnings.text.replace('\n','').replace('Please Note:','')
        except:
            warnings = 'None'
        lot_dict['Warnings'] = warnings
        for att,val in lot_dict.items():
            whiskey_dict[website].append({att:val})
        

100%|███████████████████████████████████████| 6529/6529 [43:18<00:00,  2.51it/s]


In [258]:
len(whiskey_dict)

6529

In [352]:
for i in whiskey_dict:
    print(whiskey_dict[i])
    break

[{'outer_lot_title': 'Macallan - Distil Your World - The London Edition'}, {'Highest bid': ' £7,500.00'}, {'Distillery': 'Macallan'}, {'Country': 'Scotland'}, {'Region': 'Speyside'}, {'Distillery Staus': 'Operational'}, {'Whisky Type': 'SingleMaltWhisky'}, {'Strength': '57.5%'}, {'Size': '70cl'}, {'Name': 'Macallan - Distil Your World - The London Edition'}, {'Warnings': 'None'}]


In [263]:
lst = []
for i in whiskey_dict:
    values = whiskey_dict[i]
    for j in values:
        lst.append(list(j.keys()))
        

In [266]:
lst_unique = []
for i in lst:
    if i not in lst_unique:
        lst_unique.append(i)

In [337]:
df.columns

Index(['Name', 'Link', 'Distillery', 'Vintage', 'Country', 'Region', 'Size',
       'Whisky Type', 'Bottles Produced', 'Distillery Staus', 'Strength',
       'Warnings', 'Sold on', 'Cask No.', 'Cask Finish', 'Fill Level',
       'Bottle No.'],
      dtype='object')

In [267]:
lst_unique

[['outer_lot_title'],
 ['Highest bid'],
 ['Distillery'],
 ['Country'],
 ['Region'],
 ['Distillery Staus'],
 ['Whisky Type'],
 ['Strength'],
 ['Size'],
 ['Name'],
 ['Warnings'],
 ['Winning Bid'],
 ['Sold on'],
 ['Age'],
 ['Bottles Produced'],
 ['Cask No.'],
 ['Bottle No.'],
 ['Batch No.'],
 ['Cask Type'],
 [''],
 ['Cask Finish'],
 ['Fill Level']]

In [268]:
lst.count([''])

21

In [277]:
for i in whiskey_dict:
    values = whiskey_dict[i]
    for j in values:
        for k in j:
            if k == '':
                print(j[''],i)

£0.00 https://www.highlandwhiskyauctions.com/lot-130119/teachers-large-water-jug/auction-13
£0.00 https://www.highlandwhiskyauctions.com/lot-130120/bacardi-large-water-jug/auction-13
£0.00 https://www.highlandwhiskyauctions.com/lot-130121/crawfords-small-water-jug/auction-13
£0.00 https://www.highlandwhiskyauctions.com/lot-120000/teachers-large-water-jug/auction-12
£0.00 https://www.highlandwhiskyauctions.com/lot-120018/bacardi-large-water-jug/auction-12
£0.00 https://www.highlandwhiskyauctions.com/lot-120021/crawfords-small-water-jug/auction-12
£0.00 https://www.highlandwhiskyauctions.com/lot-100444/mackinlays-water-jug/auction-10
£0.00 https://www.highlandwhiskyauctions.com/lot-100446/mackinlays-water-jug/auction-10
£0.00 https://www.highlandwhiskyauctions.com/lot-100447/ballantines-water-jug/auction-10
£0.00 https://www.highlandwhiskyauctions.com/lot-100448/ballantines-water-jug/auction-10
£0.00 https://www.highlandwhiskyauctions.com/lot-100452/highland-queen-water-jug/auction-10
£0

In [309]:
import numpy as np

In [324]:
def finder_function(link,col_name = ''):
    details = whiskey_dict[link]
    for attributes in details:
        for att,val in attributes.items():
            if att == col_name:
                return val
    return np.NaN

In [287]:
df = pd.DataFrame(columns=['Name','Link'])

In [291]:
df['Link'] = list(whiskey_dict.keys())

In [292]:
df.head()

,Name,Link
0,NaN,https://www.highlandwhiskyauctions.com/lot-130...
1,NaN,https://www.highlandwhiskyauctions.com/lot-130...
2,NaN,https://www.highlandwhiskyauctions.com/lot-130...
3,NaN,https://www.highlandwhiskyauctions.com/lot-130...
4,NaN,https://www.highlandwhiskyauctions.com/lot-130...


In [338]:
df['Name'] = df['Link'].apply(lambda x : finder_function(x,'Name'))
df['Distillery'] = df['Link'].apply(lambda x : finder_function(x,'Distillery'))
df['Vintage'] = df['Link'].apply(lambda x : finder_function(x,'Age'))
df['Country'] = df['Link'].apply(lambda x : finder_function(x,'Country'))
df['Region'] = df['Link'].apply(lambda x : finder_function(x,'Region'))
df['Size'] = df['Link'].apply(lambda x : finder_function(x,'Size'))
df['Whisky Type'] = df['Link'].apply(lambda x : finder_function(x,'Whisky Type'))
df['Bottles Produced'] = df['Link'].apply(lambda x : finder_function(x,'Bottles Produced'))
df['Distillery Staus'] = df['Link'].apply(lambda x : finder_function(x,'Distillery Staus'))
df['Strength'] = df['Link'].apply(lambda x : finder_function(x,'Strength'))
df['Warnings'] = df['Link'].apply(lambda x : finder_function(x,'Warnings'))
df['Sold on'] = df['Link'].apply(lambda x : finder_function(x,'Sold on'))
df['Cask No.'] = df['Link'].apply(lambda x : finder_function(x,'Cask No.'))
df['Cask Finish'] = df['Link'].apply(lambda x : finder_function(x,'Cask Finish'))
df['Fill Level'] = df['Link'].apply(lambda x : finder_function(x,'Fill Level'))
df['Cask Finish'] = df['Link'].apply(lambda x : finder_function(x,'Cask Finish'))
df['Bottle No.'] = df['Link'].apply(lambda x : finder_function(x,'Bottle No.'))
df['Cask Type'] = df['Link'].apply(lambda x : finder_function(x,'Cask Type'))
df['Batch No.'] = df['Link'].apply(lambda x : finder_function(x,'Batch No.'))
df['Winning Bid'] = df['Link'].apply(lambda x : finder_function(x,'Winning Bid'))
df['Highest bid'] = df['Link'].apply(lambda x : finder_function(x,'Highest bid'))

In [339]:
df.head()

,Name,Link,Distillery,Vintage,Country,Region,Size,Whisky Type,Bottles Produced,Distillery Staus,...,Warnings,Sold on,Cask No.,Cask Finish,Fill Level,Bottle No.,Cask Type,Batch No.,Winning Bid,Highest bid
0,Macallan - Distil Your World - The London Edition,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,NaN,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"£7,500.00"
1,Macallan - 30 Years Old - Sherry Oak 2022 Release,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,30YearsOld,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,None,08/02/2023,NaN,NaN,NaN,NaN,NaN,NaN,"£4,050.00",NaN
2,Springbank - 21 Year Old - First Bottle Of The...,https://www.highlandwhiskyauctions.com/lot-130...,Springbank,21YearsOld,Scotland,Campbeltown,70cl,SingleMaltWhisky,59,Operational,...,Box wear/marks.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"£4,000.00"
3,Macallan - Royal Marriage - Prince William & K...,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,NaN,Scotland,Speyside,70cl,SingleMaltWhisky,1000,Operational,...,Box & label condition.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"£3,650.00"
4,Macallan - 18 Year Old - Gran Reserva 1980,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,18YearsOld,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,None,08/02/2023,NaN,NaN,NaN,NaN,NaN,NaN,"£3,600.00",NaN


In [340]:
df['Distillery Staus'].unique()

array(['Operational', 'Closed', nan, 'Other'], dtype=object)

In [341]:
df['Whisky Type'].unique()

array(['SingleMaltWhisky', 'BlendedWhisky', 'IrishWhisky', 'IrishWhiskey',
       'Merchandise', 'BourbonWhiskey', 'Cognac', 'GrainWhisky', 'Gin',
       nan, 'Rum', 'WhiskyLiqueur', 'PureMalt', 'Brandy', 'Tequila',
       'Port', 'BourbonWhisky', 'NewMake', 'SpiritDrink'], dtype=object)

In [348]:
df['Lot Id'] = df['Link'].apply(lambda x : int(x.split('/')[3].split('-')[-1]))

In [350]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6529 entries, 0 to 6528
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              6529 non-null   object
 1   Link              6529 non-null   object
 2   Distillery        5791 non-null   object
 3   Vintage           2993 non-null   object
 4   Country           6273 non-null   object
 5   Region            5452 non-null   object
 6   Size              6306 non-null   object
 7   Whisky Type       6317 non-null   object
 8   Bottles Produced  801 non-null    object
 9   Distillery Staus  5686 non-null   object
 10  Strength          6310 non-null   object
 11  Warnings          6529 non-null   object
 12  Sold on           4541 non-null   object
 13  Cask No.          323 non-null    object
 14  Cask Finish       28 non-null     object
 15  Fill Level        1 non-null      object
 16  Bottle No.        620 non-null    object
 17  Cask Type     

In [351]:
#df.to_csv('HighLand_whisky_auction.csv')

In [353]:
df.shape

(6529, 22)

In [368]:
df['Highest bid'].isna().sum()

4562

In [385]:
def func(abc):
    if abc is np.NaN:
        return 'Not Av'
    else:
        return abc

In [389]:
df['Count_highestbid'] = df['Highest bid'].apply(lambda x : func(x))
df['Count_Winningbid'] = df['Winning Bid'].apply(lambda x : func(x))

In [390]:
df.head()

,Name,Link,Distillery,Vintage,Country,Region,Size,Whisky Type,Bottles Produced,Distillery Staus,...,Cask Finish,Fill Level,Bottle No.,Cask Type,Batch No.,Winning Bid,Highest bid,Lot Id,Count_highestbid,Count_Winningbid
0,Macallan - Distil Your World - The London Edition,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,NaN,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,NaN,NaN,NaN,NaN,NaN,NaN,"£7,500.00",130809,"£7,500.00",Not Av
1,Macallan - 30 Years Old - Sherry Oak 2022 Release,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,30YearsOld,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,NaN,NaN,NaN,NaN,NaN,"£4,050.00",NaN,130257,Not Av,"£4,050.00"
2,Springbank - 21 Year Old - First Bottle Of The...,https://www.highlandwhiskyauctions.com/lot-130...,Springbank,21YearsOld,Scotland,Campbeltown,70cl,SingleMaltWhisky,59,Operational,...,NaN,NaN,NaN,NaN,NaN,NaN,"£4,000.00",130223,"£4,000.00",Not Av
3,Macallan - Royal Marriage - Prince William & K...,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,NaN,Scotland,Speyside,70cl,SingleMaltWhisky,1000,Operational,...,NaN,NaN,NaN,NaN,NaN,NaN,"£3,650.00",130571,"£3,650.00",Not Av
4,Macallan - 18 Year Old - Gran Reserva 1980,https://www.highlandwhiskyauctions.com/lot-130...,Macallan,18YearsOld,Scotland,Speyside,70cl,SingleMaltWhisky,NaN,Operational,...,NaN,NaN,NaN,NaN,NaN,"£3,600.00",NaN,130549,Not Av,"£3,600.00"


In [373]:
'£7,500.00'.replace('£','')

'7,500.00'

In [394]:
df[(df['Count_Winningbid'] == 'Not Av') & (df['Count_highestbid'] == 'Not Av')].shape

(21, 24)

In [395]:
df.shape

(6529, 24)

In [396]:
6529-21

6508